<a href="https://colab.research.google.com/github/jugernaut/ProgramacionEnParalelo/blob/desarrollo/Envoltorios/04_TensorFlow2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<font color="Teal" face="Comic Sans MS,arial">
  <h1 align="center"><i>TensorFlow</i></h1>
  </font>
  <font color="Black" face="Comic Sans MS,arial">
  <h5 align="center"><i>Profesor: M. en C. Miguel Angel Pérez León</i></h5>
    <h5 align="center"><i>Ayudante: Jesús Iván Coss Calderón</i></h5>
    <h5 align="center"><i>Ayudante: Mario Arturo</i></h5>
  <h5 align="center"><i>Materia: Seminario de programación en paralelo</i></h5>
  </font>

## Introducción

*TensorFlow* es una *API* desarrollado por *Google* y es el conjunto de herramientas libres que se utiliza más ampliamente en el desarrollo de inteligencia artificial.

Existen multiples versiones de *TensorFlow*, sin embargo en escencia vamos a contar con la version 1.x y la versión 2.x. La principal diferencia entre ambas es que la versión 1.x hace uso de **grafos** para representar el flujo de los datos y la versión 2.x se apoya en [Keras](https://enmilocalfunciona.io/deep-learning-basico-con-keras-parte-1/) para generar modelos más intuitivos.

En este documento nos enfocaremos en la versión 2.x de *TensorFlow* con soporte para *GPU's*.

## Mapa auto organizado (*SOM*)

Un mapa auto organizado o *SOM* por sus siglas en inglés (*self-organized map*) es una de las redes neuronales más sencilla y fáciles de implementar pero no por eso es un algoritmo que no tenga aplicación actualmente.

Este tipo de red neuronal fue creado en la decada de los 80's por el por el finlandés Teuvo Kohonen y se basa en modelos matemáticos de Alan Turing.

La idea detrás de este algoritmo es muy sencilla y se describe de manera breve a continuación:

*   Comenzamos con una red o mapa (matriz) de vectores o incluso de matrices en la cual todas las neuronas o entradas del mapa contienen valores aleatorios.
*   Por cada elemento en la lista de entrenamiento, se evalua la norma (distancia) de este elemento contra cada neurona en la red.
*   Tomamos aquella neurona cuya norma haya sido la menor y modificamos los valores de las neuronas vecinas para que se parezcan un poco al vector evaluado en esa iteración.
*   Se repite este proceso hasta terminar las iteraciones ó en caso de que la norma de la red actual y la red anterior no difiere mucho.





### Formalización del algoritmo

Para dar un formalización de este algoritmo es necesario definir un conjunto de variables que son usadas durante el proceso de entrenamiento y clasificación de la red nueronal.


#### Variables

*   $s$ es la iteración actual.
*   $\lambda$ cantidad de ciclos de entrenamiento o epocas.
*   $t$ es el índice del vector de entrada en el conjunto de datos de entrada $D$.
*   $D(t)$ es un vector de entrada de índice $t$ del conjunto de datos de entrada $D$.
*   $v$ es el índice de una neurona en el mapa.
*   $W_v$ es el vector de pesos de la neurona v.
*   $u$ es el índice de la neurona cuya norma es la menor con respecto de $W_v$
*   $\Theta (u,v,s)$ es la función de vecindad que determina cuáles neuronas serán modificadas.
*   $\alpha (s)$ es una función que restringe el aprendizaje conforme avanzan las iteraciones.

#### Algoritmo

1.   Hacer un mapa (red) de neuronas con vectores de pesos aleatorios.
2.   Tomar un vector de entrada $D(t)$.

>1.   Iterar por cada neurona del mapa.

>>1.   Calcular la distancia entre el vector de entrada y los vectores de pesos de las neuronas del mapa.
2.   Mantener la neurona que ha tenido la menor distancia (norma), esta neurona será el best matching unit (BMU).

>2.   Actualizar las neuronas en la vecindad del BMU.

>> 1.   $W_{v}\left(s+1\right)=W_{v}\left(s\right)+\Theta\left(u,v,s\right)\alpha\left(s\right)\left(D\left(t\right)-W_{v}\left(s\right)\right)$

3.   Incrementar $s$ y volver al paso 2, mientras $s<\lambda$.




## *Tensor Flow*

Existen varias formas de hacer uso de *TensoFlow*, sin embargo dadas las características del curso, nos vamos a enfocar en la forma declarativa.

Lo primero que necesitamos hacer para acceder a la versión de *TensorFlow* con soporte para GPU's en Google Colab, es desinstalar la versión actual e instalar la versión con soporte para GPU's, además de cambiar el entorno de ejecución del jupyter.

1.   Para cambiar el entorno de ejecución: Primero, ir al menú *Runtime o Entorno de ejecución*, seleccionar *Cambiar tipo de tiempo de ejecución*, y en el cuadro emergente, en *Acelerador de hardware*, seleccione *GPU*, guardamos el cambio y listo.
2.   Posteriormente validamos que se tenga acceso al *GPU*.

In [14]:
import tensorflow as tf

print(tf.test.is_gpu_available())
print(tf.config.list_physical_devices('GPU'))
print(tf.__version__)

True
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
2.5.0


La celda superior nos indica que tenemos acceso al GPU's y que harémos uso de la versión 2.5.0 de *TensoFlow*.

### *FrameWork*

Inicialmente TensorFlow fue diseñado para hacer uso de grafos para representar los datos y las operaciones que se realizan sobre los mismos. Parte de esa forma de trabajar aun funciona con la versión 2.x de *TensorFlow* y es buena idea comenzar con la misma.

Como en la mayoria de *FrameWorks*, *TensorFlow* cuenta con multiples elementos que ayudan al programador, algunos de estos elementos son:


*   Constantes.
*   Variables.
*   Tensores.
*   Escalares.





#### Operaciones

Pensemos que, como parte de nuestro modelo necesitamos procesar 2 entradas y devolver un resultado. Esta operación es muy sencilla pero muestra como se debe pensar en el flujo de los datos.


In [ ]:
# se realiza la suma de 3 y 5 haciendo uso de tf y del metodo add
a = tf.add(3, 5)
# mostramos el elemento del grafo llamado a
print(a)
# se muestra el resultado de la operación en el nodo a
print(a.numpy())

tf.Tensor(8, shape=(), dtype=int32)
8


Podemos pensar en esta operación de la siguiente forma.

<center>
<img src="https://github.com/jugernaut/ProgramacionEnParalelo/blob/main/Imagenes/Envoltorios/sumaTF.png?raw=true" width="700">
</center>

#### Ventaja del grafo

El grafo nos da la ventaja de construir de manera organizada y visual la forma en la que se procesan los datos.

Ahora pensemos que deseamos realizar la siguiente operación. 

$$\left(2\times3\right)^{\left(2+5\right)}$$

¿Cómo se vería este grafo y cómo se escribe esta operación con *TensorFlow*?.

<center>
<img src="https://github.com/jugernaut/ProgramacionEnParalelo/blob/main/Imagenes/Envoltorios/powTF.png?raw=true" width="700">
</center>

In [ ]:
# Variables de Python
x = 2
y = 3

# Operaciones y grafo de TensorFlow
op1 = tf.add(x, y)         
op2 = tf.multiply(x, y)    
op3 = tf.pow(op2, op1)

# Veamos el nodo op3
print(op3)

# El resultado de dicha operación es
print(op3.numpy())

tf.Tensor(7776, shape=(), dtype=int32)
7776


#### Red Neuronal

Conforme vamos agregando más nodos al grafo, este cada vez se parece más a una red, incluso podemos llegar a un punto en el cual el grafo sea similar a una red neuronal.

<center>
<img src="https://github.com/jugernaut/ProgramacionEnParalelo/blob/main/Imagenes/Envoltorios/som.gif?raw=true" width="700">
</center>

### Acceso a la *GPU*

En la sección anterior vimos que ya se contaba con acceso a la GPU, ahora vamos a ver que tan buena idea es hacer uso de la misma.

Vamos a definir 2 mpetodos que hagan uso de TensorFlow, uno de ellos procesando los datos en la *CPU* y el otro en la *CPU*.

In [27]:
import timeit

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
    print(
        '\n\nNo se tiene habilitado el acceso a la GPU, revisa la configuracion '
        'del notebook.\n\n')
    raise SystemError('No se cuenta con GPU')

def cpu():
    # con esta linea se procesa el bloque en la CPU
    with tf.device('/cpu:0'):

      random_image_cpu = tf.random.normal((100, 100, 100))
      #net_cpu = tf.keras.layers.Conv2D(32, 7)(random_image_cpu)
      #print(random_image_cpu.numpy())
      return tf.math.reduce_sum(random_image_cpu).numpy()

def gpu():
    with tf.device('/device:GPU:0'):
      random_image_gpu = tf.random.normal((100, 100, 100))
      #net_gpu = tf.keras.layers.Conv2D(32, 7)(random_image_gpu)
      return tf.math.reduce_sum(random_image_gpu)
  
# Provemos ambos metodos
cpu()
gpu()

# Run the op several times.
print('Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images '
      '(batch x height x width x channel). Sum of ten runs.')
print('CPU (s):')
cpu_time = timeit.timeit('cpu()', number=10, setup="from __main__ import cpu")
print(cpu_time)
print('GPU (s):')
gpu_time = timeit.timeit('gpu()', number=10, setup="from __main__ import gpu")
print(gpu_time)
print('GPU speedup over CPU: {}x'.format(int(cpu_time/gpu_time)))

Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images (batch x height x width x channel). Sum of ten runs.
CPU (s):
0.15481046399986553
GPU (s):
0.003434654000102455
GPU speedup over CPU: 45x


## Extendiendo este modelo

La implementación en este documento se realizo con colores, ya que facilitan la comprensión del funcionamiento del algoritmo en general.

Sin ambargo esta red neuronal puede ser aplicada a cualquier espacio vectorial, en otras palabras, este algoritmo puede ser aplicado a cualquier objeto que podamos representar en forma de vector o matriz.

### Clasificación de Documentos o Imágenes

Para clasificar documentos el algoritmo es exactamente el mismo, lo único que cambia es que tenemos que obtener un **vector caracteristico** para los documentos que nos interes clasificar. Este vector caracteristico se puede obtener de formas muy variadas y una de ellas es **contando la frecuencia de las plabras** que aparecen en dicho documento.

Respecto a la clasificación de imágenes, una imagen finalmente es un **mapa de pixeles**, mismo que puede ser representado por un vector de vectores, es decir un **vector de colores**, lo que en si ya un vector caracteristico de dicha imágen.

## Referencias

*   https://codesachin.wordpress.com/2015/11/28/self-organizing-maps-with-googles-tensorflow/
*   http://www.saedsayad.com/clustering_som.htm
*   https://www.tensorflow.org/install
*   https://relopezbriega.github.io/blog/2016/06/05/tensorflow-y-redes-neuronales/

